In [1]:
import json
import random
from datasets import load_dataset

import warnings
warnings.filterwarnings("ignore")

DATASET_ID = "vicgalle/alpaca-gpt4"
SPLIT = "train"
OUTPUT_PATH = "alpaca_gpt4_en_pretrain.jsonl"

MAX_SAMPLES = 200_000
MAX_CHARS = 512
CHUNK_SIZE = 5000

BOS = "<|im_start|>"
EOS = "<|im_end|>"
# ============================================================


def safe_get(d, keys):
    """Return the first available key from list, or '' """
    for k in keys:
        if k in d and d[k] not in [None, ""]:
            return str(d[k])
    return ""


def build_prompt(sample):
    """
    Build a unified prompt for any dataset format.
    Supports:
      - instruction + input + output
      - instruction + output
      - text-only datasets
    """

    instr = safe_get(sample, ["instruction", "prompt", "question"])
    inp = safe_get(sample, ["input", "context"])
    out = safe_get(sample, ["output", "answer", "completion"])
    text = safe_get(sample, ["text"])

    # Raw text-only dataset (pretraining)
    if instr == "" and out == "" and text != "":
        return f"{BOS}{text}{EOS}"

    # Build user message
    user_msg = instr
    if inp != "":
        user_msg += "\n" + inp

    # No output → treat as single-turn text
    if out == "":
        return f"{BOS}{user_msg}{EOS}"

    # Instruction dataset
    return (
        f"{BOS}user\n{user_msg}{EOS}\n"
        f"{BOS}assistant\n{out}{EOS}"
    )


def write_chunk(buffer, path):
    random.shuffle(buffer)
    with open(path, "a", encoding="utf-8") as f:
        for x in buffer:
            f.write(json.dumps(x, ensure_ascii=False) + "\n")


def main():

    print(f"🚀 Loading dataset: {DATASET_ID} (streaming mode)")
    dataset = load_dataset(DATASET_ID, split=SPLIT, streaming=True)

    buffer = []
    written = 0
    total_chars = 0  # <-- track total characters

    for sample in dataset:

        if written >= MAX_SAMPLES:
            break

        prompt = build_prompt(sample)

        # Character-level filtering only
        if len(prompt) > MAX_CHARS:
            continue

        total_chars += len(prompt)

        buffer.append({"text": prompt})
        written += 1

        if len(buffer) >= CHUNK_SIZE:
            print(f"📝 Writing chunk... {written} samples so far")
            write_chunk(buffer, OUTPUT_PATH)
            buffer = []

    # final flush
    if buffer:
        write_chunk(buffer, OUTPUT_PATH)

    # final stats
    est_tokens = total_chars / 4  # approx 4 chars/token

    print("\n🎉 DONE!")
    print(f"📦 Total samples written: {written}")
    print(f"🔤 Total characters: {total_chars:,}")
    print(f"🔢 Estimated tokens: {est_tokens:,.0f}")
    print(f"🔢 Tokens (millions): {est_tokens / 1e6:.3f}M")
    print(f"🔢 Tokens (billions): {est_tokens / 1e9:.4f}B")
    print(f"📄 Saved to: {OUTPUT_PATH}")


if __name__ == "__main__":
    main()

🚀 Loading dataset: vicgalle/alpaca-gpt4 (streaming mode)
📝 Writing chunk... 5000 samples so far
📝 Writing chunk... 10000 samples so far
📝 Writing chunk... 15000 samples so far
📝 Writing chunk... 20000 samples so far

🎉 DONE!
📦 Total samples written: 23093
🔤 Total characters: 6,151,513
🔢 Estimated tokens: 1,537,878
🔢 Tokens (millions): 1.538M
🔢 Tokens (billions): 0.0015B
📄 Saved to: alpaca_gpt4_en_pretrain.jsonl
